# CONFLUENCE Tutorial 1a — Point-Scale Workflow (Paradise SNOTEL)

## Introduction

This notebook demonstrates the point-scale modeling workflow in **CONFLUENCE**, a framework for reproducible and modular computational hydrology. At the point scale, we simulate vertical energy and water fluxes at a single site, independent of routing or lateral flow, to isolate and evaluate model process representations.

Here, we focus on the **Paradise SNOTEL station (ID 602)**, located at 1,630 m elevation in Washington’s Cascade Range. This site represents a transitional snow climate and provides long-term observations of snow water equivalent (SWE) and soil moisture across multiple depths. By reproducing the observed seasonal snow and soil moisture dynamics, this tutorial demonstrates how CONFLUENCE structures a controlled, transparent, and fully reproducible point-scale experiment.

Through this example, you will see how configuration-driven workflows manage experiment setup, geospatial definition, input data preprocessing, model instantiation, and performance evaluation—building a foundation for more complex distributed modeling studies later in the series.


# Step 1 — Configuration (pick or generate)

We begin by selecting (or programmatically generating) a single configuration file that fully specifies the experiment. This keeps the workflow reproducible and makes initialization a one-liner.


In [ ]:
# Step 1 — Create a site-specific configuration for the Paradise SNOTEL example

from pathlib import Path
import yaml

# Path to the default template configuration
config_template = Path("../0_config_files/config_point_template.yaml")

# Load the base configuration
with open(config_template, "r") as f:
    config = yaml.safe_load(f)

# === Modify key entries for the Paradise SNOTEL point-scale case ===

# Define code directory — ensures relative paths resolve correctly
config["CONFLUENCE_CODE_DIR"] = str(Path("../").resolve())

# Define data directory — location of required input and observational data
config["CONFLUENCE_DATA_DIR"] = str(Path("/path/to/CONFLUENCE_data").resolve())

# Restrict the spatial domain to a single site using latitude/longitude bounds
# This ensures domain setup treats it as a point-scale (non-routed) experiment
config["DOMAIN_DEFINITION_METHOD"] = "point"
config["BOUNDING_BOX_COORDS"] = "46.781/-121.751/46.779/-121.749"
config["POUR_POINT_COORDS"] = "46.78/-121.75"

# Enable automatic download of SNOTEL data for this station
config["DOWNLOAD_SNOTEL"] = True

# Specify model and forcing dataset used in this example
config["HYDROLOGICAL_MODEL"] = "SUMMA"     # SUMMA is the process-based model used here
config["FORCING_DATASET"] = "ERA5"         # ERA5 reanalysis for meteorological inputs

# Define the temporal extent of the experiment
config["EXPERIMENT_TIME_START"] = "2000-01-01 01:00"
config["EXPERIMENT_TIME_END"] = "2002-12-31 23:00"

# Assign a descriptive domain name and experiment ID
config["DOMAIN_NAME"] = "paradise"

# === Save the customized configuration ===
out_config = Path("../0_config_files/config_paradise.yaml")
with open(out_config, "w") as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)

print(f"✅ New configuration written to: {out_config}")

## Step 1b — Initialize CONFLUENCE

With the configuration prepared, we now initialize **CONFLUENCE**.  
This step reads the configuration file, sets up the project directory, and registers all workflow managers (data, domain, model, and evaluation).  


In [ ]:
# Step 1b — Initialize CONFLUENCE

from confluence import CONFLUENCE  # adjust if your import path differs

config_path = "../0_config_files/config_paradise.yaml"
confluence = CONFLUENCE(config_path)

print("✅ CONFLUENCE initialized successfully.")
print(f"Configuration loaded from: {config_path}")

## Step 1c — Project structure setup

We now create the standardized project directory and a pour-point feature for the site.  
This anchors the experiment in a clear, reproducible file layout and records the site location for downstream domain and data steps.


In [ ]:
# Step 1c — Project structure setup

from pathlib import Path

# 1) Create the standardized project layout (logs, config link, data/output folders, etc.)
project_dir = confluence.managers['project'].setup_project()

# 2) Create a pour-point feature (the site reference geometry for point-scale workflows)
pour_point_path = confluence.managers['project'].create_pour_point()

print("✅ Project structure created.")
print(f"Project root: {project_dir}")
print(f"Pour point:   {pour_point_path}")

# 3) Brief top-level directory preview
print("\nTop-level structure:")
for p in sorted(Path(project_dir).iterdir()):
    if p.is_dir():
        print(f"├── {p.name}")


## Project Structure Creation and Organization
We can now use CONFLUENCE to setup a project directory for our experiment. 

In [ ]:
# Initialize project structure
project_dir = confluence.managers['project'].setup_project()

print(f"\n📁 Project root created: {project_dir}")

# Create spatial reference point (SNOTEL station location)
pour_point_path = confluence.managers['project'].create_pour_point()

print(f"📍 Pour point created: {pour_point_path}")
print(f"   → Location: {config_dict['POUR_POINT_COORDS']} (Paradise SNOTEL)")

# Display the created directory structure
print(f"\n=== Standardized Directory Structure ===")

def display_directory_tree(path, prefix="", max_depth=2, current_depth=0):
    """Display directory tree with scientific context"""
    if current_depth >= max_depth:
        return
    
    items = sorted([item for item in path.iterdir() if item.is_dir()])
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        # Add scientific context for key directories
        if item.name == "forcing":
            print(f"{prefix}{'    ' if is_last else '│   '}    → Meteorological input data")
        elif item.name == "observations": 
            print(f"{prefix}{'    ' if is_last else '│   '}    → Validation datasets (SNOTEL, streamflow)")
        elif item.name == "simulations":
            print(f"{prefix}{'    ' if is_last else '│   '}    → Model output organized by experiment")
        elif item.name == "attributes":
            print(f"{prefix}{'    ' if is_last else '│   '}    → Geospatial characteristics (elevation, soil, land cover)")
        elif item.name == "shapefiles":
            print(f"{prefix}{'    ' if is_last else '│   '}    → Spatial domains and discretization")
            
        if current_depth < max_depth - 1:
            extension = "    " if is_last else "│   "
            display_directory_tree(item, prefix + extension, max_depth, current_depth + 1)

display_directory_tree(project_dir, max_depth=2)

# Step 2: Geospatial Domain Definition and Spatial Discretization

## Scientific Context
Spatial representation is fundamental to hydrological modeling, determining how we conceptualize the landscape and partition it into computational units. The choice of spatial discretization profoundly affects:

- Process Representation: How we capture spatial heterogeneity in climate, topography, vegetation, and soils
- Model Complexity: The trade-off between process detail and computational efficiency
- Scale Dependencies: How processes manifest differently at point, hillslope, and watershed scales
- Validation Strategy: What observations are appropriate for model evaluation

For point-scale modeling, we deliberately minimize spatial complexity to isolate vertical processes. This creates a controlled environment where energy and water balance physics can be evaluated without the confounding effects of lateral flow, spatial heterogeneity, or routing processes.

The spatial representation we establish here contrasts with the distributed watersheds we'll explore in Tutorial 2, where complex topography drives spatial patterns in precipitation, radiation, and runoff generation.

## CONFLUENCE Implementation
CONFLUENCE handles spatial domain definition through three components:

- Attribute Acquisition: Systematic collection of the geospatial characteristics we need to configure our hydrological models (elevation, soil properties, land cover) using standardized datasets
- Domain Delineation: Creation of the primary computational boundary (Grouped Response Units - GRUs)
- Domain Discretization: Subdivision into Hydrologic Response Units (HRUs) based on landscape similarity

For point-scale studies, this process creates a minimal spatial representation:

- Bounding Box: 0.001° × 0.001° square centered on station coordinates
- Single GRU: One computational unit representing the station footprint
- Single HRU: No further subdivision needed for point-scale physics

The same framework scales seamlessly from this minimal representation to complex distributed watersheds with hundreds of HRUs as we'll explore in Tutorials 2 and 3.

## Step 2a: Geospatial Attribute Acquisition
Attributes provide the physical characteristics needed to parameterize model physics. Even for point-scale modeling, we need elevation, soil properties, and vegetation characteristics to constrain energy and water balance processes.

CONFLUENCE uses [gistool (Keshavaraz et al., 2025](https://github.com/CH-Earth/gistool) to subset and aquire the required data.

In [ ]:
# Acquire geospatial attributes
print(f"\n⬇️  Acquiring attributes through gistool (Model Agnostic Framework)...")
print("   → This may take several minutes ")

# confluence.managers['data'].acquire_attributes()

print("✅ Attribute acquisition complete")    

## Step 2b: Domain Delineation (GRU Creation)
Domain delineation creates the primary computational boundary. For point-scale modeling, this is simply a geometric square around our station location, but the same process scales to complex watershed delineation for distributed modeling.

In [ ]:
print("Creating primary computational boundary...")

watershed_path = confluence.managers['domain'].define_domain()

print(f"✅ Domain delineation complete")

## Step 2c: Domain Discretization (HRU Creation)
Discretization subdivides GRUs into Hydrologic Response Units based on landscape similarity. For point-scale modeling, we maintain a 1:1 relationship (1 GRU = 1 HRU), but this step demonstrates the framework that enables more complex spatial representations which we will encounter in later notebook sections.

In [ ]:
print("Creating Hydrologic Response Units for model execution...")

hru_path = confluence.managers['domain'].discretize_domain()

print(f"✅ Domain discretization complete")

## Spatial Visualization

In [ ]:
# Load spatial data
hru_path = confluence.project_dir / "shapefiles" / "catchment" / f"{config_dict['DOMAIN_NAME']}_HRUs_{config_dict['DOMAIN_DISCRETIZATION']}.shp"
hru_gdf = gpd.read_file(hru_path)

# Create simple plot
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# Plot HRU
hru_gdf.plot(ax=ax, facecolor='lightblue', edgecolor='blue', linewidth=2, alpha=0.7)

# Add station point
station_coords = config_dict['POUR_POINT_COORDS'].split('/')
station_lat, station_lon = float(station_coords[0]), float(station_coords[1])
ax.scatter(station_lon, station_lat, c='red', s=100, marker='*', 
          label=f'Paradise SNOTEL\n({station_lat:.4f}, {station_lon:.4f})', zorder=5)

# Styling
ax.set_title(f'Point-Scale Domain: {config_dict["DOMAIN_NAME"].title()}', fontsize=14, fontweight='bold')
ax.set_xlabel('Longitude (°)', fontsize=12)
ax.set_ylabel('Latitude (°)', fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

# Add annotation
ax.text(0.02, 0.98, f"Area: ~0.012 km²\n1 GRU, 1 HRU\nPoint-scale representation", 
       transform=ax.transAxes, fontsize=10, verticalalignment='top',
       bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.3'))

plt.tight_layout()
plt.show()

print("✅ Spatial visualization complete")

# Step 3: Input Data Preprocessing and Model-Agnostic Framework
## Scientific Context
Input data preprocessing represents a critical but often overlooked component of hydrological modeling that profoundly affects model performance and scientific conclusions. Traditional approaches tightly couple data preprocessing with specific models, creating several scientific and practical challenges:

- Model Comparison Barriers: Different preprocessing approaches make it difficult to determine whether performance differences arise from model physics or data preparation
- Reproducibility Issues: Model-specific preprocessing pipelines are often poorly documented and difficult to reproduce
- Research Inefficiency: Duplicated preprocessing effort across modeling studies
- Benchmarking Limitations: Inability to evaluate models against consistent baselines

The Model-Agnostic Framework (MAF) philosophy addresses these challenges by separating data preparation from model execution, creating a standardized pipeline that serves multiple modeling applications.

## CONFLUENCE Implementation Philosophy
CONFLUENCE implements a two-stage preprocessing architecture:
Stage 1: Model-Agnostic Preprocessing

- Standardized Data Sources: Consistent meteorological and geospatial datasets
- Unified Spatial Framework: Common geospatial operations across all models
- Quality-Controlled Outputs: Standardized formats with documented provenance
- Reusable Products: Same preprocessed data serves multiple models and analyses

Stage 2: Model-Specific Preprocessing

- Format Translation: Convert standardized outputs to model-required formats
- Model Configuration: Apply model-specific parameter assignments and settings
- Initialization: Prepare model-specific initial conditions and control files

This separation enables true model intercomparison studies, automated benchmarking, and scalable research workflows that maintain scientific rigor while maximizing efficiency.

## Step 3a: Meteorological Forcing Data Acquisition
Meteorological forcing drives all hydrological models, making standardized acquisition critical for reproducible research. CONFLUENCE leverages the Model-Agnostic Framework's [datatool (Keshavarz et al., 2025)](https://github.com/CH-Earth/datatool) to access quality-controlled, globally-consistent datasets.

In [ ]:
print("Acquiring standardized meteorological forcing through datatool...")

# confluence.managers['data'].acquire_forcings()

print("✅ Forcing data acquisition complete")

## Step 3b: Observational Data Processing
Observational data provides the ground truth for model evaluation. CONFLUENCE systematically acquires and processes multiple observation types, creating standardized validation datasets that support comprehensive model assessment.

In [ ]:
# Execute observational data processing
print(f"\n📥 Processing observational datasets...")
confluence.managers['data'].process_observed_data()

print("✅ Observational data processing complete")

## Step 3c: Model-Agnostic Preprocessing Pipeline
The model-agnostic preprocessing represents the core innovation of CONFLUENCE's data management philosophy. This stage creates standardized, model-independent data products that serve as the foundation for all subsequent modeling activities.

In [ ]:
print(f"\n⚙️  Executing model-agnostic preprocessing...")

confluence.managers['data'].run_model_agnostic_preprocessing()

print("✅ Model-agnostic preprocessing complete")

## Step 3d: Model-Specific Preprocessing
Model-specific preprocessing translates the standardized model-agnostic products into the formats and configurations required by individual models. This stage maintains the scientific benefits of standardized inputs while accommodating diverse model requirements.

Remapping of the forcing data and zonal statistics calculations for the geospatial attributes is performed in one model-agnostic pre-processing step. 

In [ ]:
print(f"\n🔧 Executing SUMMA-specific preprocessing...")

confluence.managers['model'].preprocess_models()

print("✅ Model-specific preprocessing complete")

# Step 4: Model Instantiation and Process-Based Simulation
## Scientific Context
Model instantiation represents the critical transition from static data preparation to dynamic process simulation. This step transforms spatially-distributed inputs and temporally-varying forcing into evolving hydrological states through the explicit representation of physical processes.
In process-based hydrological modeling, we solve coupled differential equations representing:

- Energy Balance: Net radiation partitioning between sensible, latent, and ground heat fluxes
- Water Balance: Precipitation partitioning among interception, infiltration, evapotranspiration, and runoff
- Snow Physics: Accumulation, metamorphism, and melt processes with explicit energy considerations
- Soil Hydrology: Infiltration, redistribution, and drainage through layered soil profiles
- Vegetation Dynamics: Canopy interception, transpiration, and phenological controls

The SUMMA (Structure for Unifying Multiple Modeling Alternatives) framework enables systematic evaluation of process representations, making it ideal for scientific hypothesis testing and model physics assessment.

## CONFLUENCE Implementation
CONFLUENCE manages model execution through several integrated components:

- Workflow Orchestration: Automated sequencing of model initialization, spinup, and main simulation
- Configuration Management: Translation of scientific decisions into model-specific control files
- Execution Monitoring: Real-time tracking of model progress and error detection
- Output Organization: Systematic storage and cataloging of simulation results
- Quality Assurance: Automated checks for mass balance closure and physical realism

This framework ensures that model execution is reproducible, traceable, and scientifically rigorous, while handling the computational complexity behind the scenes.

In [ ]:
print(f"\nRunning {confluence.config['HYDROLOGICAL_MODEL']} for point-scale simulation...")

confluence.managers['model'].run_models()

print("\nPoint-scale model run complete")

# Step 5: Model Evaluation and Process Validation
## Scientific Context
Model evaluation represents the critical test of whether our process-based simulation captures the essential physics of the real-world system. Effective evaluation requires:

- Multi-Variable Assessment: Testing multiple aspects of the hydrological system to avoid equifinality and ensure robust process representation
- Temporal Pattern Analysis: Evaluating both magnitude and timing of hydrological responses across seasonal cycles
- Process-Specific Metrics: Using evaluation criteria that reflect the underlying physics being tested
- Uncertainty Quantification: Understanding both observational and model uncertainty in performance assessment

For point-scale modeling, we focus on direct process validation where observations closely match the spatial and temporal scales of model representation. The Paradise SNOTEL station provides co-located snow water equivalent and multi-depth soil moisture observations.

In [ ]:
# Load simulation data
sim_dir = confluence.project_dir / "simulations" / config_dict['EXPERIMENT_ID'] / "SUMMA"
daily_output_path = sim_dir / f"{config_dict['EXPERIMENT_ID']}_day.nc"

# Load and prepare the evaluation dataset
ds = xr.open_dataset(daily_output_path)

# Skip spinup period
start_year = ds.time.dt.year.min().values + 1
spinup_end = f"{start_year}-01-01"
time_mask = ds.time >= pd.to_datetime(spinup_end)
evaluation_data = ds.isel(time=time_mask)

# Load observed SWE data
obs_swe_path = confluence.project_dir / "observations" / "snow" / "snotel" / "processed" / f"{config_dict['DOMAIN_NAME']}_swe_processed.csv"

obs_swe = pd.read_csv(obs_swe_path, parse_dates=['Date'], dayfirst=True)
obs_swe.set_index('Date', inplace=True)

# Ensure proper datetime index
if not isinstance(obs_swe.index, pd.DatetimeIndex):
    obs_swe.index = pd.to_datetime(obs_swe.index)

# Extract simulated SWE
sim_swe = evaluation_data['scalarSWE'].to_pandas()

# Find common period and align data
start_date = max(obs_swe.index.min(), sim_swe.index.min())
end_date = min(obs_swe.index.max(), sim_swe.index.max())

# Resample to daily and filter to common period
obs_daily = obs_swe.resample('D').mean().loc[start_date:end_date]
sim_daily = sim_swe.resample('D').mean().loc[start_date:end_date]

# Handle different column names for SWE
if 'SWE' in obs_daily.columns:
    obs_values = obs_daily['SWE']
elif 'swe' in obs_daily.columns:
    obs_values = obs_daily['swe']
else:
    # Use first column
    obs_values = obs_daily.iloc[:, 0]

# Convert sim_daily to Series if it's a DataFrame
if isinstance(sim_daily, pd.DataFrame):
    if len(sim_daily.columns) == 1:
        sim_daily = sim_daily.iloc[:, 0]  # Extract the single column as Series

# Remove NaN values for metrics calculation
valid_mask = ~(obs_values.isna() | sim_daily.isna())
obs_valid = obs_values[valid_mask]
sim_valid = sim_daily[valid_mask]

print(f"\n📊 Snow Water Equivalent Performance Metrics:")

# Basic statistics
rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
bias = (sim_valid - obs_valid).mean()
mae = np.abs(obs_valid - sim_valid).mean()
corr = obs_valid.corr(sim_valid)

# Percent bias
pbias = 100 * bias / obs_valid.mean()

# Nash-Sutcliffe Efficiency
nse = 1 - ((obs_valid - sim_valid) ** 2).sum() / ((obs_valid - obs_valid.mean()) ** 2).sum()

# Kling-Gupta Efficiency
kge_corr = obs_valid.corr(sim_valid)
kge_bias = sim_valid.mean() / obs_valid.mean()
kge_var = (sim_valid.std() / obs_valid.std())
kge = 1 - np.sqrt((kge_corr - 1)**2 + (kge_bias - 1)**2 + (kge_var - 1)**2)

# Display metrics
print(f"   Root Mean Square Error (RMSE): {rmse:.2f} mm")
print(f"   Mean Absolute Error (MAE): {mae:.2f} mm")
print(f"   Bias: {bias:+.2f} mm ({pbias:+.1f}%)")
print(f"   Correlation: {corr:.3f}")
print(f"   Nash-Sutcliffe Efficiency: {nse:.3f}")
print(f"   Kling-Gupta Efficiency: {kge:.3f}")

# Snow-specific metrics
print(f"\n❄️  Snow-Specific Performance Assessment:")

# Peak SWE analysis
obs_peak = obs_valid.max()
sim_peak = sim_valid.max()
peak_bias = sim_peak - obs_peak
peak_pbias = 100 * peak_bias / obs_peak

print(f"       Peak SWE:")
print(f"       Observed: {obs_peak:.1f} mm")
print(f"       Simulated: {sim_peak:.1f} mm")
print(f"       Bias: {peak_bias:+.1f} mm ({peak_pbias:+.1f}%)")

# Snow season timing
obs_peak_date = obs_valid.idxmax()
sim_peak_date = sim_valid.idxmax()
timing_diff = (sim_peak_date - obs_peak_date).days

print(f"       Peak Timing:")
print(f"       Observed peak: {obs_peak_date.strftime('%B %d, %Y')}")
print(f"       Simulated peak: {sim_peak_date.strftime('%B %d, %Y')}")
print(f"       Timing difference: {timing_diff:+d} days")

# Snow season length
snow_threshold = 10  # mm
obs_snow_days = (obs_valid > snow_threshold).sum()
sim_snow_days = (sim_valid > snow_threshold).sum()

print(f"       Snow Season (SWE > {snow_threshold} mm):")
print(f"       Observed: {obs_snow_days} days")
print(f"       Simulated: {sim_snow_days} days")
print(f"       Difference: {sim_snow_days - obs_snow_days:+d} days")

# Create  visualization
print(f"\n Creating SWE comparison visualization...")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time series plot
ax1 = axes[0, 0]
ax1.plot(obs_daily.index, obs_values, 'o-', label='Observed', 
         color='black', alpha=0.7, markersize=3, linewidth=1)
ax1.plot(sim_daily.index, sim_daily, '-', label='Simulated', 
         color='blue', linewidth=2)
ax1.set_title('Snow Water Equivalent Time Series', fontsize=12, fontweight='bold')
ax1.set_ylabel('SWE (mm)', fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.legend()

# Add performance metrics as text
metrics_text = f'RMSE: {rmse:.1f} mm\nBias: {bias:+.1f} mm\nCorr: {corr:.3f}\nNSE: {nse:.3f}'
ax1.text(0.02, 0.95, metrics_text, transform=ax1.transAxes, 
         bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')

# Scatter plot
ax2 = axes[0, 1]
ax2.scatter(obs_valid, sim_valid, alpha=0.6, c='blue', s=20)
max_val = max(obs_valid.max(), sim_valid.max())
ax2.plot([0, max_val], [0, max_val], 'k--', label='1:1 line')
ax2.set_xlabel('Observed SWE (mm)', fontsize=11)
ax2.set_ylabel('Simulated SWE (mm)', fontsize=11)
ax2.set_title('Observed vs. Simulated SWE', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_aspect('equal', adjustable='box')

# Seasonal cycle
ax3 = axes[1, 0]
obs_monthly = obs_values.groupby(obs_values.index.month).mean()
sim_monthly = sim_daily.groupby(sim_daily.index.month).mean()
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax3.plot(range(1, 13), obs_monthly, 'o-', label='Observed', color='black', linewidth=2)
ax3.plot(range(1, 13), sim_monthly, 'o-', label='Simulated', color='blue', linewidth=2)
ax3.set_xticks(range(1, 13))
ax3.set_xticklabels(months, rotation=45)
ax3.set_ylabel('Mean SWE (mm)', fontsize=11)
ax3.set_title('Seasonal Cycle', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.legend()

# Residuals over time
ax4 = axes[1, 1]
residuals = sim_valid - obs_valid
ax4.scatter(obs_valid.index, residuals, alpha=0.6, c='red', s=15)
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax4.axhline(y=residuals.std(), color='red', linestyle='--', alpha=0.5, label=f'±1σ ({residuals.std():.1f} mm)')
ax4.axhline(y=-residuals.std(), color='red', linestyle='--', alpha=0.5)
ax4.set_ylabel('Residuals (Sim - Obs) [mm]', fontsize=11)
ax4.set_title('Model Residuals Over Time', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.legend()

plt.suptitle(f'Snow Water Equivalent Evaluation - {config_dict["DOMAIN_NAME"].title()}', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Step 5b: Soil Moisture Profile Evaluation
Soil moisture evaluation tests the model's representation of vadose zone processes, including infiltration, drainage, and vertical redistribution. Multi-depth observations provide unprecedented validation opportunities for soil physics.

In [ ]:
# Load observed soil moisture data
obs_sm_path = confluence.project_dir / "observations" / "soil_moisture" / "ismn" / "processed" / f"{config_dict['DOMAIN_NAME']}_sm_processed.csv"

# Load observed data
obs_sm = pd.read_csv(obs_sm_path, parse_dates=['timestamp'])
obs_sm.set_index('timestamp', inplace=True)

# Ensure proper datetime index
if not isinstance(obs_sm.index, pd.DatetimeIndex):
    obs_sm.index = pd.to_datetime(obs_sm.index)

# Identify observed depth columns
obs_depth_cols = [col for col in obs_sm.columns if col.startswith('sm_')]

# Extract observed depth values
obs_depths = []
for depth_col in obs_depth_cols:
    # Extract depth from column name (e.g., 'sm_0.0508_0.0508' -> 0.0508)
    depth_str = depth_col.split('_')[1]
    obs_depths.append(float(depth_str))

# Extract simulated soil moisture

sim_sm = evaluation_data['mLayerVolFracLiq']
sim_depths = evaluation_data['mLayerDepth']
    
# Calculate representative layer depths
mean_layer_depths = sim_depths.mean(dim='time')
valid_layers = mean_layer_depths > 0  # Filter out invalid layers
    
# Find common period
start_date = max(obs_sm.index.min(), pd.to_datetime(sim_sm.time.min().values))
end_date = min(obs_sm.index.max(), pd.to_datetime(sim_sm.time.max().values))
    
# Filter to common period
obs_period = obs_sm.loc[start_date:end_date]
sim_time_mask = (sim_sm.time >= start_date) & (sim_sm.time <= end_date)
sim_period = sim_sm.isel(time=sim_time_mask)
sim_depths_period = sim_depths.isel(time=sim_time_mask)
    
n_depths = len(obs_depth_cols)
depth_results = {}

# Analyze each observed depth
for i, (depth_col, obs_depth) in enumerate(zip(obs_depth_cols, obs_depths)):
    print(f"\n     Depth {i+1}: {obs_depth:.4f}m ({depth_col})")
    
    # Find closest simulated layer
    mean_depths = sim_depths_period.mean(dim='time')
    valid_mask = mean_depths > 0
    
    if valid_mask.sum() > 0:
        valid_mean_depths = mean_depths.where(valid_mask)
        depth_differences = np.abs(valid_mean_depths - obs_depth)
        closest_layer_idx = depth_differences.argmin().values
        closest_layer_depth = valid_mean_depths[closest_layer_idx].values
        
        
        # Extract data for this layer
        obs_layer = obs_period[depth_col]
        sim_layer = sim_period.isel(midToto=closest_layer_idx, hru=0)
        
        # Convert to pandas for easier handling
        sim_layer_ts = sim_layer.to_pandas()
        
        # Resample to daily and align
        obs_daily = obs_layer.resample('D').mean()
        sim_daily = sim_layer_ts.resample('D').mean()
        
        # Remove invalid values (negative soil moisture indicates missing data)
        sim_daily = sim_daily.where(sim_daily > -100)
        
        # Find valid paired data
        valid_data_mask = ~(obs_daily.isna() | sim_daily.isna())
        obs_valid = obs_daily[valid_data_mask]
        sim_valid = sim_daily[valid_data_mask]
        
        if len(obs_valid) > 10:  # Require minimum data for meaningful evaluation
            # Calculate performance metrics
            rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
            bias = (sim_valid - obs_valid).mean()
            mae = np.abs(obs_valid - sim_valid).mean()
            corr = obs_valid.corr(sim_valid)
            
            # Store results
            depth_results[obs_depth] = {
                'obs_valid': obs_valid,
                'sim_valid': sim_valid,
                'rmse': rmse,
                'bias': bias,
                'mae': mae,
                'corr': corr,
                'layer_idx': closest_layer_idx,
                'sim_depth': closest_layer_depth
            }
            
            print(f"       RMSE: {rmse:.3f} m³/m³")
            print(f"       Bias: {bias:+.3f} m³/m³")
            print(f"       Correlation: {corr:.3f}")
            print(f"       Valid pairs: {len(obs_valid)}")
            
# Create  visualization
print(f"\n  Creating soil moisture profile evaluation...")

n_depths = len(depth_results)
fig, axes = plt.subplots(n_depths, 2, figsize=(15, 4*n_depths))

if n_depths == 1:
    axes = axes.reshape(1, -1)

for i, (obs_depth, results) in enumerate(depth_results.items()):
    # Time series plot
    ax_ts = axes[i, 0]
    ax_ts.plot(results['obs_valid'].index, results['obs_valid'], 'o-', 
              label=f'Observed ({obs_depth:.4f}m)', 
              color='black', alpha=0.7, markersize=2, linewidth=1)
    ax_ts.plot(results['sim_valid'].index, results['sim_valid'], '-', 
              label=f'Simulated (L{results["layer_idx"]}, {results["sim_depth"]}m)', 
              color='blue', linewidth=2)
    
    ax_ts.set_title(f'Soil Moisture at {obs_depth:.4f}m depth', fontsize=11, fontweight='bold')
    ax_ts.set_ylabel('Soil Moisture (m³/m³)', fontsize=10)
    ax_ts.grid(True, alpha=0.3)
    ax_ts.legend(fontsize=9)
    
    # Add metrics
    metrics_text = (f"RMSE: {results['rmse']:.3f}\n"
                   f"Bias: {results['bias']:+.3f}\n"
                   f"Corr: {results['corr']:.3f}")
    ax_ts.text(0.02, 0.95, metrics_text, transform=ax_ts.transAxes,
              bbox=dict(facecolor='white', alpha=0.8), fontsize=9, verticalalignment='top')
    
    # Scatter plot
    ax_scatter = axes[i, 1]
    ax_scatter.scatter(results['obs_valid'], results['sim_valid'], 
                      alpha=0.6, c='blue', s=15)
    
    # 1:1 line
    min_val = min(results['obs_valid'].min(), results['sim_valid'].min())
    max_val = max(results['obs_valid'].max(), results['sim_valid'].max())
    ax_scatter.plot([min_val, max_val], [min_val, max_val], 'k--', 
                   label='1:1 line', alpha=0.7)
    
    ax_scatter.set_xlabel('Observed SM (m³/m³)', fontsize=10)
    ax_scatter.set_ylabel('Simulated SM (m³/m³)', fontsize=10)
    ax_scatter.set_title(f'Obs vs Sim at {obs_depth:.4f}m', fontsize=11, fontweight='bold')
    ax_scatter.grid(True, alpha=0.3)
    ax_scatter.legend(fontsize=9)
    ax_scatter.set_aspect('equal', adjustable='box')

plt.suptitle(f'Soil Moisture Profile Evaluation - {config_dict["DOMAIN_NAME"].title()}', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Summary performance assessment
print(f"\n  Soil Moisture Profile Performance Summary:")

avg_rmse = np.mean([r['rmse'] for r in depth_results.values()])
avg_corr = np.mean([r['corr'] for r in depth_results.values()])
avg_bias = np.mean([r['bias'] for r in depth_results.values()])

print(f"       Profile-averaged metrics:")
print(f"       Average RMSE: {avg_rmse:.3f} m³/m³")
print(f"       Average correlation: {avg_corr:.3f}")
print(f"       Average bias: {avg_bias:+.3f} m³/m³")

# Close evaluation dataset
evaluation_data.close()

# Tutorial Summary and Next Steps

## Summary: Point-Scale Snow and Soil Process Evaluation

This tutorial demonstrated the complete CONFLUENCE workflow for point-scale hydrological modeling, establishing fundamental principles for reproducible computational hydrology research. Through the Paradise SNOTEL case study, we illustrated how standardized workflows can bridge the gap between complex modeling frameworks and practical scientific applications while maintaining scientific rigor and reproducibility.

### Key Methods
The tutorial successfully demonstrated CONFLUENCE's core capabilities through five integrated workflow components. Reproducible workflow management was achieved through configuration-driven experiments that provide complete provenance tracking and ensure experimental transparency. The model-agnostic preprocessing pipeline creates standardized data products that enable true model physics comparisons by separating data preparation from model-specific requirements. Process-based simulation was implemented using SUMMA's modular physics to explicitly represent energy and water balance processes at the point scale. Multi-variable validation leveraged co-located snow water equivalent and multi-depth soil moisture observations to provide comprehensive model evaluation. Finally, scientific interpretation linked quantitative performance metrics to underlying physical processes, moving beyond statistical assessment to process understanding.

### Scientific Process Validation
The tutorial validated key hydrological processes across multiple temporal scales and physical domains. Snow physics evaluation included accumulation, metamorphism, and energy-balance driven melt processes, with assessment of both magnitude and timing accuracy. Soil hydrology validation examined multi-depth moisture dynamics and vertical water redistribution through four soil layers, testing infiltration and drainage representations. Energy balance processes were implicitly evaluated through successful simulation of temperature-driven phase changes and moisture dynamics. Temporal dynamics were assessed from daily to seasonal time scales, capturing both rapid response and longer-term memory effects. Physical realism was maintained through mass and energy conservation checks and realistic state variable bounds.

### CONFLUENCE Framework Demonstration
This tutorial showcased CONFLUENCE's modular architecture through specialized managers that handle distinct workflow components while maintaining system integration. Workflow orchestration capabilities were demonstrated through automated step sequencing. The scalable design principle was established, showing how the same framework structure supports applications from point-scale studies through continental-scale analyses. Research continuity was ensured by creating a foundation that directly enables progression to distributed modeling and large-sample comparative studies in subsequent tutorials.

### Next Focus: Evapotranspiration and Energy Balance Processes

**Ready to explore energy flux validation?** → **[Tutorial 01b: Point-Scale Energy Balance Validation](./01b_point_scale_fluxnet.ipynb)**